In [ ]:
%load_ext autoreload
%autoreload 1
%aimport aymurai.datasets.utils

In [ ]:
!sudo wget -c -P /usr/share/tesseract-ocr/4.00/tessdata https://github.com/tesseract-ocr/tessdata/blob/4767ea922bcc460e70b87b1d303ebdfed0897da8/spa.traineddata

In [ ]:
import numpy as np
import pandas as pd
import spacy
from spacy import displacy

import locale

locale.setlocale(locale.LC_ALL, "es_AR.UTF-8")

In [ ]:

import os
import re
import numpy as np
from glob import iglob

from datasets import Dataset

from aymurai.text.extraction import extract_document



def __load_doc(example: dict) -> dict:
    example["text"] = extract_document(example["path"], method='tesseract', language='spa', errors='raise')
    return example


def registro(example: dict) -> dict:
    filename = os.path.basename(example['path'])
    filename, ext = os.path.splitext(filename)
    
    example['doc_name'] = filename
    example['doc_extension'] = ext

    dirname = os.path.dirname(example['path'])
    match = re.findall('/(\d{4})/', dirname)
    example['year'] = match[0] if match else None

    filename = re.sub(r'\s', '_', filename)
    filename = re.sub(r'^[A-Z]\d+_', '', filename)
    example['nro_registro'] = filename.split('_')[0]

    tomo = filename.split('_')[1]
    match = re.findall('\d+', tomo)
    example['tomo'] = match[0] if match else None

    return example


def load_documents_from(path: str) -> Dataset:
    files = iglob(f"{path}/**/*.*", recursive=True)
    files = filter(os.path.isfile, files)
    files = list(sorted(files))

    dataset = Dataset.from_dict({"path": files})
    dataset = dataset.map(registro, load_from_cache_file=True, desc='registro')
    dataset = dataset.map(__load_doc, desc="Loading documents")
    dataset = dataset.filter(lambda example: bool(example["text"]))

    return dataset


In [ ]:
private = load_documents_from('/resources/restricted/ar-juz-pcyf-10/RESOLUCIONES DEL JUZGADO-pdf/')


In [ ]:

df = private.to_pandas()
df.dropna(subset=['nro_registro', 'tomo', 'year'], inplace=True)
df['nro_registro'] = df['nro_registro'].astype(str)
df['year'] = df['year'].astype(int)
df['tomo'] = df['tomo'].astype(int)
df

In [ ]:
db = pd.read_csv('/resources/data/preprocessed.csv')
db

In [ ]:
db.merge(df, on=['nro_registro', 'tomo'])